In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_function as theodo_f
import matplotlib.pyplot as plt
import random
random.seed(0)

In [ ]:
P1 = np.array([[1,0,0,0.5],[0,1,0,-0.25],[0,0,1,0.3],[0,0,0,1]])
P2 = np.array([[1,0,0,-0.1],[0,1,0,0.4],[0,0,1,0.2],[0,0,0,1]])
P3 = np.array([[1,0,0,-0.7],[0,1,0,0],[0,0,1,0.4],[0,0,0,1]])

# Distance in meter
Dist_12 = np.linalg.norm(np.array([P1[0,3],P1[1,3],P1[2,3]])-np.array([P2[0,3],P2[1,3],P2[2,3]]))
Dist_13 = np.linalg.norm(np.array([P1[0,3],P1[1,3],P1[2,3]])-np.array([P3[0,3],P3[1,3],P3[2,3]]))
Dist_23 = np.linalg.norm(np.array([P2[0,3],P2[1,3],P2[2,3]])-np.array([P3[0,3],P3[1,3],P3[2,3]]))

In [ ]:
theodo_u = importlib.reload(theodo_u)
# Trajectories simulator (3D)
# Timestamps
begin = 0  #s
end = 60  #s               
rate = 1000  #Hz           
t = np.arange(begin, end, 1/rate)
xt = np.arange(len(t))

# Speed functions
# Traj 1
#speed_linear = np.sin(t)+1
#speed_yaw = np.sin(0.5*t)*0.08
#speed_pitch = np.sin(t)*0.01
#speed_roll = np.sin(t)*0.01

# Traj 2
speed_linear = np.ones_like(xt)
speed_yaw = np.sin(0.5*t)*0
speed_pitch = np.sin(t)*0
speed_roll = np.sin(t)*0

# Traj 3
#speed_linear = np.sin(0.3*t)*np.cos(0.5*t)+1
#speed_yaw = np.sin(0.15*t)*0.3
#speed_pitch = np.sin(0.6*t)*0.01
#speed_roll = np.sin(0.1*t)*0.01

T_3d, yaw, pitch, roll, x, y, z = theodo_u.traj_simulation(t, speed_linear, speed_yaw, speed_pitch, speed_roll)
T_arr = np.array(T_3d)

In [ ]:
%matplotlib notebook
fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, 1)
#plt.subplot(211)
ax0.scatter(t,T_arr[:,0,3], s=1, marker="*")
ax0.set_xlabel('Time [s]')
ax0.set_ylabel('X [m]')
#plt.subplot(212)
ax1.scatter(t,T_arr[:,1,3], s=1, marker="*")
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Y [m]')

ax2.scatter(t,T_arr[:,2,3], s=1, marker="*")
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Z [m]')

ax3.scatter(T_arr[:,0,3],T_arr[:,1,3], s=1, marker="*")
ax3.set_xlabel('X [m]')
ax3.set_ylabel('Y [m]')

#plt.scatter(T_arr[:,0,3],T_arr[:,2,3], s=1, marker="*")
#plt.scatter(t,T_arr[:,2,3], s=1, marker="*")
#plt.plot(t,yaw)
#plt.plot(t,pitch)
#plt.plot(t,roll)
#plt.plot(t,speed_yaw)
#plt.plot(t,x)
#plt.plot(t,y)
#plt.legend(loc='best')
fig.tight_layout()
plt.axis('equal')
plt.show()
# plt.savefig("Traj_2_reference.jpg")

In [ ]:
theodo_u = importlib.reload(theodo_u)

mode = "Sync"  #Async or Sync
mean_e = 0
std_e = 0.00
mean_g = 0
std_g = 0.00
mean_noise_t = 0
std_noise_t = 0
rate = 2.5

P1_ref, P2_ref ,P3_ref, P1_noise, P2_noise, P3_noise, P1_noise_sub, P2_noise_sub , P3_noise_sub, t_sub_1, t_sub_2, t_sub_3, e_noise, t_noise_1, t_noise_2, t_noise_3, t_subref_1, t_subref_2, t_subref_3, P1_wnoise_sub, P2_wnoise_sub, P3_wnoise_sub, d1, d2, d3 = theodo_u.noise_apply(mode, mean_e, std_e, mean_g, std_g, mean_noise_t, std_noise_t, rate, T_arr, t, P1, P2, P3)

In [ ]:
P = np.array([P1[0:4, 3], P2[0:4, 3], P3[0:4, 3]]).T
d1 = []
d2 = []
d3 = []
for i in range(0, len(T_arr)):
    # Reference
    Pr = (T_arr[i] @ P)[0:3,:].T
    p1_ref = Pr[0]
    p2_ref = Pr[1]
    p3_ref = Pr[2]
    d1.append(abs(np.linalg.norm(p1_ref - p2_ref))*1000)
    d2.append(abs(np.linalg.norm(p1_ref - p3_ref)) * 1000)
    d3.append(abs(np.linalg.norm(p2_ref - p3_ref)) * 1000)
    
#for i,j,k,l in zip(T_arr, P1_noise, P2_noise, P3_noise):
#    # Reference
#    j = np.hstack((np.array(j), np.array([1]))).T
#    k = np.hstack((np.array(k), np.array([1]))).T
#    l = np.hstack((np.array(l), np.array([1]))).T
#    p1_ref = (i @ j)[0:3].T
#    p2_ref = (i @ k)[0:3].T
#    p3_ref = (i @ l)[0:3].T
#    d1.append(abs(np.linalg.norm(p1_ref - p2_ref))*1000)
#    d2.append(abs(np.linalg.norm(p1_ref - p3_ref)) * 1000)
#    d3.append(abs(np.linalg.norm(p2_ref - p3_ref)) * 1000)

In [ ]:
print("Mean [mm]: ", np.mean(abs(d1-Dist_12*1000)),np.mean(abs(d2-Dist_13*1000)),np.mean(abs(d3-Dist_23*1000)))
print("Std [mm]: ", np.std(abs(d1-Dist_12*1000)),np.std(abs(d2-Dist_13*1000)),np.std(abs(d3-Dist_23*1000)))
#print("Mean [mm]: ", np.mean(d1*1000),np.mean(d2*1000),np.mean(d3*1000))
#print("Std [mm]: ", np.std(d1*1000),np.std(d2*1000),np.std(d3*1000))
plt.plot(abs(d1-Dist_12*1000), label = "r12")
plt.plot(abs(d2-Dist_13*1000), label = "r13")
plt.plot(abs(d3-Dist_23*1000), label = "r23")
#plt.plot(d1*1000, label = "r12")
#plt.plot(d2*1000, label = "r13")
#plt.plot(d3*1000, label = "r23")
ax = plt.gca()
ax.set_yscale('symlog')
plt.legend(loc='best')
plt.show()

In [ ]:
%matplotlib notebook
T_sub_arr_1 = np.array(t_sub_1)
T_sub_arr_2 = np.array(t_sub_2)
T_sub_arr_3 = np.array(t_sub_3)
P1_n_sub = np.array(P1_noise_sub)
P2_n_sub = np.array(P2_noise_sub)
P3_n_sub = np.array(P3_noise_sub)
P1_wnoise_subr = np.array(P1_wnoise_sub)
P2_wnoise_subr = np.array(P2_wnoise_sub)
P3_wnoise_subr = np.array(P3_wnoise_sub)

# plt.scatter(T_sub_arr_1,T_sub_arr_1, marker="*")
# plt.scatter(T_sub_arr_2,T_sub_arr_2, marker="*")
# plt.scatter(T_sub_arr_3,T_sub_arr_3, marker="*")
# plt.scatter(P1_n_sub[:,0],P1_n_sub[:,1], s=1, marker="*", label="p1")
# plt.scatter(P2_n_sub[:,0],P2_n_sub[:,1], s=1, marker="*", label="p2")
# plt.scatter(P3_n_sub[:,0],P3_n_sub[:,1], s=1, marker="*", label="p3")
plt.scatter(P1_wnoise_subr[:,0],P1_wnoise_subr[:,1], s=1, marker="*", label="p1")
plt.scatter(P2_wnoise_subr[:,0],P2_wnoise_subr[:,1], s=1, marker="*", label="p2")
plt.scatter(P3_wnoise_subr[:,0],P3_wnoise_subr[:,1], s=1, marker="*", label="p3")
#plt.scatter(T_arr[:,0,3],T_arr[:,1,3], s=1, marker="*")
plt.legend(loc='best')
plt.axis('equal')
plt.show()

In [ ]:
d12_ref = []
d13_ref = []
d23_ref = []
for i,j,k in zip(P1_ref, P2_ref ,P3_ref):
    p1 = np.array([i[0],i[1],i[2]])
    p2 = np.array([j[0],j[1],j[2]])
    p3 = np.array([k[0],k[1],k[2]])
    d12_ref.append(abs(np.linalg.norm(p1-p2))*1000)
    d13_ref.append(abs(np.linalg.norm(p1-p3))*1000)
    d23_ref.append(abs(np.linalg.norm(p2-p3))*1000)
    
d12_sub = []
d13_sub = []
d23_sub = []
for i,j,k in zip(P1_wnoise_subr, P2_wnoise_subr ,P3_wnoise_subr):
    p1 = np.array([i[0],i[1],i[2]])
    p2 = np.array([j[0],j[1],j[2]])
    p3 = np.array([k[0],k[1],k[2]])
    d12_sub.append(abs(np.linalg.norm(p1-p2))*1000)
    d13_sub.append(abs(np.linalg.norm(p1-p3))*1000)
    d23_sub.append(abs(np.linalg.norm(p2-p3))*1000)

d12_n = []
d13_n = []
d23_n = []
for i,j,k in zip(P1_noise, P2_noise, P3_noise):
    p1 = np.array([i[0],i[1],i[2]])
    p2 = np.array([j[0],j[1],j[2]])
    p3 = np.array([k[0],k[1],k[2]])
    d12_n.append(abs(np.linalg.norm(p1-p2))*1000)
    d13_n.append(abs(np.linalg.norm(p1-p3))*1000)
    d23_n.append(abs(np.linalg.norm(p2-p3))*1000)

In [ ]:
print("Mean [mm]: ", np.mean(abs(d12_ref-Dist_12*1000)),np.mean(abs(d13_ref-Dist_13*1000)),np.mean(abs(d23_ref-Dist_23*1000)))
print("Std [mm]: ", np.std(abs(d12_ref-Dist_12*1000)),np.std(abs(d13_ref-Dist_13*1000)),np.std(abs(d23_ref-Dist_23*1000)))
plt.plot(d12_ref-Dist_12*1000, label = "r12")
plt.plot(d13_ref-Dist_13*1000, label = "r13")
plt.plot(d23_ref-Dist_23*1000, label = "r23")
# plt.plot(t_subref_1, d12_sub-Dist_12, label = "s12")
# plt.plot(t_subref_2, d13_sub-Dist_13, label = "s13")
# plt.plot(t_subref_3, d23_sub-Dist_23, label = "s23")
# print("Mean [mm]: ", np.mean(abs(d12_sub-Dist_12*1000)),np.mean(abs(d13_sub-Dist_13*1000)),np.mean(abs(d23_sub-Dist_23*1000)))
# print("Std [mm]: ", np.std(abs(d12_sub-Dist_12*1000)),np.std(abs(d13_sub-Dist_13*1000)),np.std(abs(d23_sub-Dist_23*1000)))
#plt.plot(d12_n-Dist_12, label = "n12")
#plt.plot(d13_n-Dist_13, label = "n13")
#plt.plot(d23_n-Dist_23, label = "n23")
# print("Mean [mm]: ", np.mean(abs(d12_n-Dist_12*1000)),np.mean(abs(d13_n-Dist_13*1000)),np.mean(abs(d23_n-Dist_23*1000)))
# print("Std [mm]: ", np.std(abs(d12_n-Dist_12*1000)),np.std(abs(d13_n-Dist_13*1000)),np.std(abs(d23_n-Dist_23*1000)))
ax = plt.gca()
ax.set_yscale('symlog')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(np.array(t_subref_1)-np.array(t_sub_1), label = "1")
plt.plot(np.array(t_subref_2)-np.array(t_sub_2), label = "2")
plt.plot(np.array(t_subref_3)-np.array(t_sub_3), label = "3")
plt.legend(loc='best')
plt.show()

In [ ]:
theodo_u = importlib.reload(theodo_u)
path = "../data/Simulation_1/"
sub_path = "total_stations/"
if(mode=="Sync"):
    mode_s = "sync"
elif(mode=="Async"):
    mode_s = "async"

#Reference pose
theodo_u.Convert_datapr_to_csv(t, T_arr, path + sub_path +  "ref" + ".csv")

# Reference points
theodo_u.Convert_raw_data_point_to_csv(t, P1_ref, path + sub_path +  "ref_p_1" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t, P2_ref, path + sub_path +  "ref_p_2" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t, P3_ref, path + sub_path +  "ref_p_3" + ".csv")

# Noise only points
theodo_u.Convert_raw_data_point_to_csv(t, P1_noise, path + sub_path + "no_p_1" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t, P2_noise, path + sub_path + "no_p_2" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t, P3_noise, path + sub_path + "no_p_3" + ".csv")

# Subsampling no noises points 
theodo_u.Convert_raw_data_point_to_csv(t_subref_1, P1_wnoise_subr, path + sub_path + "s_" + mode_s + "_p_1" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t_subref_2, P2_wnoise_subr, path + sub_path + "s_" + mode_s + "_p_2" +  ".csv")
theodo_u.Convert_raw_data_point_to_csv(t_subref_3, P3_wnoise_subr, path + sub_path + "s_" + mode_s + "_p_3" +  ".csv")

# Noise and subsampling points 
theodo_u.Convert_raw_data_point_to_csv(t_sub_1, P1_noise_sub, path + sub_path + "ns_" + mode_s + "_p_1" + ".csv")
theodo_u.Convert_raw_data_point_to_csv(t_sub_2, P2_noise_sub, path + sub_path + "ns_" + mode_s + "_p_2" +  ".csv")
theodo_u.Convert_raw_data_point_to_csv(t_sub_3, P3_noise_sub, path + sub_path + "ns_" + mode_s + "_p_3" +  ".csv")

# Noise saved
theodo_u.Convert_datane_to_csv(e_noise, path + sub_path + "ne_" + mode_s + "_p_1" + ".csv")
theodo_u.Convert_datant_to_csv(t_noise_1, t_noise_2, t_noise_3, path + sub_path + "nt_" + mode_s + "_p_1" + ".csv")